In [1]:
import os
from tqdm import tqdm
import pandas as pd
import mido
from mido import MidiFile, MidiTrack, Message, MetaMessage
import numpy as np

In [2]:
MDBS_MIDI = {
    "KD": 35,
    "SD": 38,
    "SDB": 38,
    "SDD": 38,
    "SDF": 38,
    "SDG": 38,
    "SDNS": 38,
    "CHH": 42,
    "OHH": 46,
    "PHH": 44,
    "HIT": 50,
    "MHT": 48,
    "HFT": 43,
    "LFT": 41,
    "RDC": 51,
    "RDB": 53,
    "CRC": 49,
    "CHC": 52,
    "SPC": 55,
    "SST": 37,
    "TMB": 54, 
}

In [3]:
mix_folder = 'D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/audio/full_mix/'
mixes = os.listdir(mix_folder)
mixes = [mix_folder + m for m in mixes]

In [4]:
drum_folder = 'D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/audio/drum_only/'
drum = os.listdir(drum_folder)
drum = [drum_folder + d for d in drum]

In [5]:
beats_folder = 'D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/annotations/beats/'
beats = os.listdir(beats_folder)
beats = [beats_folder + b for b in beats]

In [6]:
class_folder = 'D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/annotations/subclass/'
classes = os.listdir(class_folder)
classes = [class_folder + c for c in classes]

In [7]:
for idx, val in tqdm(enumerate(classes)):

    name = val.replace('D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/annotations/subclass/', '')
    name = name.replace('_subclass.txt', '')

    f = open(beats[idx], "r")
    b = f.read()

    f = open(classes[idx], "r")
    c = f.read()
    

    bets = b.replace('\n', '	').split('	')
    bs = []
    for idx, val in enumerate(bets):
        if idx % 2 == 0:
            bs.append(val)
    
    bs = bs[:len(bs) - 1]
    bs = [float(b) for b in bs]

    clsses = c.replace("\n", "\t").replace(" ", "")
    clsses = clsses.split("\t")
    times = []
    cl = []
    for idx, val in enumerate(clsses):
        if idx % 2 == 0:
            times.append(val)
        else:
            cl.append(val)
    
    if len(times) != len(cl):
        times = times[:len(times) - 1]
    
    times = [float(t) for t in times]
    
    clsses = cl
    times = times
    bets = bs
    # Create a new MIDI file and a MIDI track for drums
    mid = MidiFile()
    drum_track = MidiTrack()
    mid.tracks.append(drum_track)
    
    # Set a default tempo (120 BPM as an example)
    tempo = mido.bpm2tempo(120)
    drum_track.append(MetaMessage('set_tempo', tempo=tempo))
    
    # In General MIDI, channel 10 (0-indexed as 9) is reserved for percussion
    # and does not typically require a program change.
    # However, if setting a program number for a specific setup, here's how:
    program_number = 0  # Example: Standard Acoustic Grand Piano
    drum_track.append(Message('program_change', program=program_number, channel=9, time=0))
    
    # Process annotations and create MIDI events
    last_time = 0.0
    for cls, time in zip(clsses, times):
        note = MDBS_MIDI[cls]
        # Calculate delta time in ticks
        delta_time_seconds = time - last_time
        ticks_per_beat = mid.ticks_per_beat
        delta_ticks = int(mido.second2tick(delta_time_seconds, ticks_per_beat, tempo))
        # Add MIDI note on event to the percussion channel
        drum_track.append(Message('note_on', note=note, velocity=64, time=delta_ticks, channel=9))
        # Immediate note off event for simplicity
        drum_track.append(Message('note_off', note=note, velocity=64, time=0, channel=9))
        last_time = time
    
    # Save the MIDI file
    midi_file_name = f'D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/midi/{name}.mid'
    mid.save(midi_file_name)


23it [00:00, 185.50it/s]
